In [2]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import time
from confluent_kafka import Producer, Consumer
import socket
import os

HOSTNAME = "localhost"
os.makedirs("recu", exist_ok=True)

Adresse IP publique du serveur Kafka
listeners=PLAINTEXT://<votre_ip_publique>:9092

Publicité de l'adresse du broker pour les clients externes
advertised.listeners=PLAINTEXT://<votre_ip_publique>:9092

# Producer :

In [ ]:
# Configuration du Producer avec compression
conf = {
    'bootstrap.servers': HOSTNAME + ":9092",
    'client.id': socket.gethostname(),
    'compression.type': 'gzip',  # Compression type: gzip, snappy, lz4, or zstd
    'message.max.bytes': 500000000  # Increase if necessary
}
producer = Producer(conf)

# Fonction pour charger un fichier en bytes
def send_file(file_path, topic):
    try:
        with open(file_path, 'rb') as file:
            file_bytes = file.read()  # Lire le fichier en bytes
            
            # Vérifier si la taille dépasse les limites du message
            if len(file_bytes) > conf.get('message.max.bytes', 1048576):  # 1 MB par défaut
                raise ValueError("File size exceeds Kafka message size limit even with compression.")
            
            producer.produce(topic,key=file_path[6:], value=file_bytes)
            print(f"File {file_path} sent successfully to topic {topic}")
    except Exception as e:
        print(f"Failed to send file: {e}")
    finally:
        producer.flush()

# Envoi d'une image compressée automatiquement par Kafka
send_file('envoi/2.png', 'files_topic')

# Consummer :

In [ ]:
# Configuration du Consumer
conf = {
    'bootstrap.servers': HOSTNAME + ":9092",
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)

# Souscription au topic
topic = 'files_topic'
consumer.subscribe([topic])

# vra global
output_path = ''

# Fonction pour recevoir et sauvegarder un fichier
def receive_file():
    global output_path
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue
            output_path = 'recu/'+msg.key().decode('utf-8')
            # Sauvegarder les bytes reçus comme un fichier
            with open(output_path, 'wb') as file:
                file.write(msg.value())
                print(f"File received and saved to {output_path}")
    except KeyboardInterrupt:
        print("Consumer interrupted.")
    finally:
        consumer.close()

# Réception de l'image
receive_file()

# Dedection

In [ ]:
model = YOLO('yolov8n.pt')


if output_path[-4:] in [".mp4", ".mkv", ".avi", ".mov", ".wmv", ".flv", ".webm", 
    ".mpg", ".mpeg", ".3gp", ".ts", ".ogg", ".rm"]:
    
    # cap = cv2.VideoCapture(0)  # For Webcam
    #cap.set(3, 1280)
    #cap.set(4, 720)
    cap = cv2.VideoCapture(output_path)  # For Video
    
    classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
                "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                "teddy bear", "hair drier", "toothbrush"
                ]
    
    prev_frame_time = 0
    new_frame_time = 0
    
    while True:
        new_frame_time = time.time()
        success, img = cap.read()
        results = model(img, stream=True)
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Bounding Box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
                w, h = x2 - x1, y2 - y1
                cvzone.cornerRect(img, (x1, y1, w, h))
                # Confidence
                conf = math.ceil((box.conf[0] * 100)) / 100
                # Class Name
                cls = int(box.cls[0])
    
                cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)
    
        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        print(fps)
    
        cv2.imshow("Image", img)
        cv2.waitKey(1)
elif output_path[-4:] in [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".tif", 
    ".webp", ".heif", ".heic", ".raw", ".svg", ".ico", ".eps"]:
    results = model(output_path, show=True)
    cv2.waitKey(0)

# Detection webcam :

In [3]:
producer_conf = {
    'bootstrap.servers': HOSTNAME + ":9092",
    'client.id': socket.gethostname(),
    'compression.type': 'gzip',
    'message.max.bytes': 500000000  # Adjust as needed
}
producer = Producer(producer_conf)

# Create directories
capture_dir = "captured_images"
received_dir = "received_images"

os.makedirs(capture_dir, exist_ok=True)
os.makedirs(received_dir, exist_ok=True)

# Function to capture images
def capture_images(num_images=5):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Unable to access webcam.")
        return
    for i in range(num_images):
        ret, frame = cap.read()
        if ret:
            filename = os.path.join(capture_dir, f"image_{i}.jpg")
            cv2.imwrite(filename, frame)
            print(f"Captured and saved: {filename}")
        else:
            print("Failed to capture an image.")
    cap.release()
    cv2.destroyAllWindows()

# Function to send a file to Kafka
def send_file_to_kafka(file_path, topic='files_topic'):
    try:
        with open(file_path, 'rb') as file:
            file_bytes = file.read()
            if len(file_bytes) > producer_conf.get('message.max.bytes', 1048576):
                raise ValueError("File size exceeds Kafka message size limit.")
            producer.produce(topic, key=file_path.split("/")[-1], value=file_bytes)
            print(f"File {file_path} sent to topic {topic}.")
    except Exception as e:
        print(f"Error sending file to Kafka: {e}")
    finally:
        producer.flush()

In [4]:
# Capture images
capture_images(5)

# Send captured images
for filename in os.listdir(capture_dir):
    file_path = os.path.join(capture_dir, filename)
    send_file_to_kafka(file_path)

Captured and saved: captured_images\image_0.jpg
Captured and saved: captured_images\image_1.jpg
Captured and saved: captured_images\image_2.jpg
Captured and saved: captured_images\image_3.jpg
Captured and saved: captured_images\image_4.jpg
File captured_images\image_0.jpg sent to topic files_topic.
File captured_images\image_1.jpg sent to topic files_topic.
File captured_images\image_2.jpg sent to topic files_topic.
File captured_images\image_3.jpg sent to topic files_topic.
File captured_images\image_4.jpg sent to topic files_topic.


In [6]:
# Initialize YOLO model
model = YOLO('yolov8n.pt')

# Function to process with YOLO
def process_with_yolo(file_path):
    if file_path.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Process single image
        results = model(file_path, show=True)
        cv2.waitKey(0)
    elif file_path.lower().endswith(('.mp4', '.avi', '.mkv')):
        # Process video
        cap = cv2.VideoCapture(file_path)
        prev_frame_time = 0
        new_frame_time = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            new_frame_time = time.time()
            results = model(frame, stream=True)
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = round(box.conf[0], 2)
                    cls = int(box.cls[0])
                    class_name = model.names[cls]
                    cvzone.putTextRect(frame, f'{class_name} {conf}', (x1, y1))
                    cvzone.cornerRect(frame, (x1, y1, x2 - x1, y2 - y1))

            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time
            print(f"FPS: {fps}")
            cv2.imshow("YOLO Prediction", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
# Kafka Consumer Configuration
consumer_conf = {
    'bootstrap.servers': HOSTNAME + ":9092",
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(consumer_conf)

# Function to consume files from Kafka
def receive_files_from_kafka(topic='files_topic'):
    consumer.subscribe([topic])
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue
            file_name = msg.key().decode('utf-8').split("\\")[-1]
            file_path = os.path.join(received_dir, file_name)
            with open(file_path, 'wb') as file:
                file.write(msg.value())
                print(f"File saved: {file_path}")
            process_with_yolo(file_path)  # Pass the file to YOLO
    except KeyboardInterrupt:
        print("Consumer stopped.")
    finally:
        consumer.close()

# Start consuming
receive_files_from_kafka()


File saved: received_images\image_1.jpg



image 1/1 c:\Users\kamal\Desktop\Project_2 _kafka\received_images\image_1.jpg: 480x640 1 person, 86.9ms
Speed: 2.3ms preprocess, 86.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)



File saved: received_images\image_2.jpg


image 1/1 c:\Users\kamal\Desktop\Project_2 _kafka\received_images\image_2.jpg: 480x640 1 person, 64.6ms
Speed: 1.6ms preprocess, 64.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



File saved: received_images\image_3.jpg


...

File saved: received_images\image_3.jpg


image 1/1 c:\Users\kamal\Desktop\Project_2 _kafka\received_images\image_3.jpg: 480x640 1 person, 66.6ms
Speed: 1.0ms preprocess, 66.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)



File saved: received_images\image_4.jpg


image 1/1 c:\Users\kamal\Desktop\Project_2 _kafka\received_images\image_4.jpg: 480x640 1 person, 67.0ms
Speed: 3.0ms preprocess, 67.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


Consumer stopped.
